<a href="https://colab.research.google.com/github/yogeshdhome/Coursera-IBM-AI-ML-Course/blob/master/DL0321EN_4_1_Comparing_Models_py_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
#Download the data

#!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [2]:
#Unzip the folder
#!unzip concrete_data_week4.zip

In [3]:
import requests, zipfile, io

In [4]:
url = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip'

r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("/content/")

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [5]:
pip install git+git://github.com/fchollet/keras.git --upgrade --no-deps

  Cloning git://github.com/fchollet/keras.git to /tmp/pip-req-build-fsbx8gt5
  Running command git clone -q git://github.com/fchollet/keras.git /tmp/pip-req-build-fsbx8gt5
ERROR: File "setup.py" not found for legacy project git+git://github.com/fchollet/keras.git.


In [6]:
# Import libraries,modules and packages
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [7]:
#Check number of images
import os
print("Positive training samples - ",len(os.listdir('concrete_data_week4/train/positive')))
print("Negative training samples - ",len(os.listdir('concrete_data_week4/train/negative')))
print("Positive validation samples - ",len(os.listdir('concrete_data_week4/valid/positive')))
print("Negative validation samples - ",len(os.listdir('concrete_data_week4/valid/negative')))
print("Positive test samples - ",len(os.listdir('concrete_data_week4/test/positive')))
print("Negative test samples - ",len(os.listdir('concrete_data_week4/test/negative')))

Positive training samples -  15001
Negative training samples -  15002
Positive validation samples -  4751
Negative validation samples -  4752
Positive test samples -  250
Negative test samples -  250


In [8]:
#Check sample image size
from PIL import Image
import matplotlib.pyplot as plt
print("Train image size",Image.open('concrete_data_week4/train/negative/'+ os.listdir('concrete_data_week4/train/negative')[0]).size)

Train image size (227, 227)


In [9]:
#Global constants, Image size need to be 224*224 to match with VGG model 
num_classes = 2
image_resize = 224
batch_size_training = 100
batch_size_validation = 100

In [10]:
#Instantiate ImageDataGenerator with pre-trained VGG16 model
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [11]:
#Train generator
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [12]:
#Valid generator
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [13]:
#Instantiat Sequential model object
model = Sequential()

In [14]:
#Add VGG16 model(Pre-trained) as hidden layer
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

58892288/58889256 [==============================] - 0s 0us/step


In [15]:
#Output layer
model.add(Dense(num_classes, activation='softmax'))

In [16]:
model.layers

In [17]:
model.layers[0].trainable = False

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [19]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [21]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
301/301 [==============================] - 218s 577ms/step - loss: 0.0873 - accuracy: 0.9707 - val_loss: 0.0245 - val_accuracy: 0.9954
Epoch 2/2
301/301 [==============================] - 177s 587ms/step - loss: 0.0184 - accuracy: 0.9958 - val_loss: 0.0150 - val_accuracy: 0.9968


In [22]:
#model.save('/content/classifier_vgg_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [39]:
resnet_model = tensorflow.keras.models.load_model("classifier_resnet_model.h5")

In [52]:
#Test generator

test_generator = data_generator.flow_from_directory(
    '/content/concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle = False)

Found 500 images belonging to 2 classes.


In [53]:
print("VGG16 model Accuracy:", model.evaluate_generator(test_generator)[1],
      "\nVGG16 model loss:",model.evaluate_generator(test_generator)[0])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


VGG16 model Accuracy: 0.9959999918937683 
VGG16 model loss: 0.012727946043014526


In [54]:
print("ResNet model Accuracy:", resnet_model.evaluate_generator(test_generator)[1],
      "\nResnet model loss:",resnet_model.evaluate_generator(test_generator)[0])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


ResNet model Accuracy: 1.0 
Resnet model loss: 0.0023061400279402733


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [59]:
for i in range(248,253,1):
  class_0,class_1 = model.predict_generator(test_generator)[i]
  if class_0 > class_1:
    print("\nNegative")
  else:
    print("\nPositive")


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '



Negative

Negative

Positive

Positive

Positive


In [60]:
for i in range(5):
  class_0,class_1 = resnet_model.predict_generator(test_generator)[i]
  if class_0 > class_1:
    print("\nNegative")
  else:
    print("\nPositive")


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '



Negative

Negative

Negative

Negative

Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).